In [66]:
# %pip install flet plotly pandas


In [67]:
# import yfinance

In [68]:
# import flet as ft
# import sqlite3
# import pandas as pd
# import plotly.graph_objs as go

# DB_PATH = "Data/G2_Mercado.db"

# def carregar_acoes():
#     conn = sqlite3.connect(DB_PATH)
#     df = pd.read_sql_query("SELECT sigla FROM Acao ORDER BY sigla", conn)
#     conn.close()
#     return df["sigla"].tolist()

# def carregar_dados(sigla, data_ini, data_fim):
#     conn = sqlite3.connect(DB_PATH)
#     query = """
#         SELECT data, abertura, fechamento, maxima, minima, volume
#         FROM HistoricoAcao
#         WHERE sigla = ? AND data BETWEEN ? AND ?
#         ORDER BY data
#     """
#     df = pd.read_sql_query(query, conn, params=(sigla, data_ini, data_fim))
#     conn.close()
#     df["data"] = pd.to_datetime(df["data"])
#     return df

# def main(page: ft.Page):
#     page.title = "Análise de Ações - Flet"
#     page.scroll = ft.ScrollMode.AUTO

#     dropdown = ft.Dropdown(label="Selecione a ação")
#     date_ini = ft.TextField(label="Data Início (YYYY-MM-DD)", width=150)
#     date_fim = ft.TextField(label="Data Fim (YYYY-MM-DD)", width=150)
#     btn = ft.ElevatedButton("Gerar Gráficos")

#     aba_conteudo = ft.Tabs(expand=1, tabs=[])

#     def gerar_graficos(e):
#         aba_conteudo.tabs.clear()
#         df = carregar_dados(dropdown.value, date_ini.value, date_fim.value)
#         if df.empty:
#             page.snack_bar = ft.SnackBar(ft.Text("Nenhum dado encontrado."))
#             page.snack_bar.open = True
#             page.update()
#             return

#         fig1 = go.Figure(data=[go.Scatter(x=df["data"], y=df["fechamento"], name="Fechamento")])
#         fig1.update_layout(title="Preço de Fechamento")

#         fig2 = go.Figure(data=[go.Bar(x=df["data"], y=df["volume"], name="Volume")])
#         fig2.update_layout(title="Volume Diário")

#         media = df["fechamento"].rolling(7).mean()
#         fig3 = go.Figure()
#         fig3.add_trace(go.Scatter(x=df["data"], y=df["fechamento"], name="Fechamento"))
#         fig3.add_trace(go.Scatter(x=df["data"], y=media, name="Média Móvel"))
#         fig3.update_layout(title="Média Móvel de Fechamento")

#         fig4 = go.Figure()
#         fig4.add_trace(go.Scatter(x=df["data"], y=df["maxima"], name="Máxima", line=dict(color="green")))
#         fig4.add_trace(go.Scatter(x=df["data"], y=df["minima"], name="Mínima", line=dict(color="red")))
#         fig4.update_layout(title="Máxima e Mínima Diária")

#         aba_conteudo.tabs.extend([
#             ft.Tab(text="Fechamento", content=ft.PlotlyChart(fig1)),
#             ft.Tab(text="Volume", content=ft.PlotlyChart(fig2)),
#             ft.Tab(text="Média Móvel", content=ft.PlotlyChart(fig3)),
#             ft.Tab(text="Máx/Mín", content=ft.PlotlyChart(fig4))
#         ])
#         page.update()

#     btn.on_click = gerar_graficos

#     dropdown.options = [ft.dropdown.Option(sigla) for sigla in carregar_acoes()]
#     page.add(ft.Row([dropdown, date_ini, date_fim, btn], alignment="center"))
#     page.add(aba_conteudo)

# ft.app(target=main)


In [69]:
# import yfinance as yf
# dat = yf.Ticker("PETR4.SA")

In [70]:
# dat.history(period='1mo')

In [82]:
import sqlite3
import yfinance as yf
from datetime import datetime, timedelta
import os

# Configurações de caminho
DB_PATH = os.path.join("Data", "G2_Mercado.db")
ACOES_PATH = os.path.join("Data", "Acoes.txt")
NAO_CARREGADAS_PATH = os.path.join("Data", "NaoCarregadas.txt")
# Criar arquivo de ações (se não existir)
if not os.path.exists(ACOES_PATH):
    with open(ACOES_PATH, "w") as f:
        f.write("\n".join([
            "PETR4.SA", "VALE3.SA", "ITUB4.SA",
            "BBDC4.SA", "ABEV3.SA", "BBAS3.SA"
        ]))
# Conectar ao banco
conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()
# Ativar foreign keys
cursor.execute("PRAGMA foreign_keys = ON")
# Criar tabelas
cursor.execute('''
CREATE TABLE IF NOT EXISTS Acao (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    sigla TEXT UNIQUE,
    nome TEXT
)
''')
cursor.execute('''
CREATE TABLE IF NOT EXISTS HistoricoAcao (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    data DATE,
    abertura REAL,
    fechamento REAL,
    maxima REAL,
    minima REAL,
    volume INTEGER,
    acao_id INTEGER,
    FOREIGN KEY (acao_id) REFERENCES Acao(id) ON DELETE CASCADE
)
''')
# Limpar dados antigos
cursor.execute("DELETE FROM HistoricoAcao")
cursor.execute("DELETE FROM Acao")
conn.commit()
# Ler ações
with open(ACOES_PATH, "r") as f:
    acoes = [linha.strip() for linha in f if linha.strip()]
# Datas
inicio = datetime.now().date() - timedelta(days=360)
fim = datetime.now().date()
nao_carregadas = []
# Coleta de dados
for sigla in acoes:
    try:
        ticker = yf.Ticker(sigla)
        nome = ticker.info.get("longName", "Desconhecido")
        cursor.execute("INSERT INTO Acao (sigla, nome) VALUES (?, ?)", (sigla, nome))
        acao_id = cursor.lastrowid

        df = ticker.history(start=inicio.isoformat(), end=fim.isoformat())
        for data, row in df.iterrows():
            cursor.execute("""
                INSERT INTO HistoricoAcao
                (data, abertura, fechamento, maxima, minima, volume, acao_id)
                VALUES (?, ?, ?, ?, ?, ?, ?)
            """, (
                data.date().isoformat(),
                row["Open"],
                row["Close"],
                row["High"],
                row["Low"],
                int(row["Volume"]),
                acao_id
            ))
        conn.commit()

    except Exception:
        nao_carregadas.append(sigla)
        conn.rollback()

# Registrar ações não carregadas
with open(NAO_CARREGADAS_PATH, "w") as f:
    f.write("\n".join(nao_carregadas))

conn.close()
print("Carga concluída com sucesso.")

Carga concluída com sucesso.


In [1]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Box(
    y=[0.75, 5.25, 5.5, 6, 6.2, 6.6, 6.80, 7.0, 7.2, 7.5, 7.5, 7.75, 8.15,
       8.15, 8.65, 8.93, 9.2, 9.5, 10, 10.25, 11.5, 12, 16, 20.90, 22.3, 23.25],
    name="All Points",
    jitter=0.3,
    pointpos=-1.8,
    boxpoints='all', # represent all points
    marker_color='rgb(7,40,89)',
    line_color='rgb(7,40,89)'
))

fig.add_trace(go.Box(
    y=[0.75, 5.25, 5.5, 6, 6.2, 6.6, 6.80, 7.0, 7.2, 7.5, 7.5, 7.75, 8.15,
        8.15, 8.65, 8.93, 9.2, 9.5, 10, 10.25, 11.5, 12, 16, 20.90, 22.3, 23.25],
    name="Only Whiskers",
    boxpoints=False, # no data points
    marker_color='rgb(9,56,125)',
    line_color='rgb(9,56,125)'
))

fig.add_trace(go.Box(
    y=[0.75, 5.25, 5.5, 6, 6.2, 6.6, 6.80, 7.0, 7.2, 7.5, 7.5, 7.75, 8.15,
        8.15, 8.65, 8.93, 9.2, 9.5, 10, 10.25, 11.5, 12, 16, 20.90, 22.3, 23.25],
    name="Suspected Outliers",
    boxpoints='suspectedoutliers', # only suspected outliers
    marker=dict(
        color='rgb(8,81,156)',
        outliercolor='rgba(219, 64, 82, 0.6)',
        line=dict(
            outliercolor='rgba(219, 64, 82, 0.6)',
            outlierwidth=2)),
    line_color='rgb(8,81,156)'
))

fig.add_trace(go.Box(
    y=[0.75, 5.25, 5.5, 6, 6.2, 6.6, 6.80, 7.0, 7.2, 7.5, 7.5, 7.75, 8.15,
        8.15, 8.65, 8.93, 9.2, 9.5, 10, 10.25, 11.5, 12, 16, 20.90, 22.3, 23.25],
    name="Whiskers and Outliers",
    boxpoints='outliers', # only outliers
    marker_color='rgb(107,174,214)',
    line_color='rgb(107,174,214)'
))


fig.update_layout(title_text="Box Plot Styling Outliers")
fig.show()

In [ ]:
fig.write_image("figure.png")